In [2]:
import gym
import or_gym
from pyomo.environ import *
from or_gym.algos.math_prog_utils import *
import numpy as np

In [61]:
env_name = 'Knapsack-v2'
env = gym.make(env_name)
env.item_values = np.array([10, 5])
env.item_probs = np.array([0.75, 0.25])
env.item_weights = np.array([2, 1])
env.N = 2
env.item_numbers = np.arange(env.N)
env.step_limit = 2

# Initialize model
m = ConcreteModel()

# Sets, parameters, and variables
m.W = env.max_weight
m.T = env.step_limit

m.i = Set(initialize=env.item_numbers)
m.t = RangeSet(0, m.T - 1) # Time steps
m.s = Set(initialize=env.item_numbers) # Scenarios

m.w = Param(m.i, 
    initialize={i: j for i, j in zip(env.item_numbers, env.item_weights)})
m.v = Param(m.i, 
    initialize={i: j for i, j in zip(env.item_numbers, env.item_values)})
m.p = Param(m.i,
    initialize={i: j for i, j in zip(env.item_numbers, env.item_probs)})

m.x = Var(m.i, m.t, m.s, within=Binary)

@m.Constraint()
def weight_constraint(m):
    return sum(m.w[i] * m.x[i, t, s] 
               for i in m.i
               for t in m.t 
               for s in m.s) - m.W <= 0

@m.Constraint(m.t, m.s)
def assignment_constraint(m, t, s):
    return sum(m.x[i, t, s] 
               for i in m.i
               for s in m.s) <= 1

@m.Constraint(m.i, m.t, m.s)
def scenario_constraints(m, i, t, s):
    if i == s:
        return (m.x[i, t, s] <= 1)
    else:
        return (m.x[i, t, s] == 0)


m.obj = Objective(expr=(
    sum([m.v[i] * m.x[i, t, s] 
         for i in m.i
         for t in m.t
         for s in m.s])),
    sense=maximize)

In [62]:
model, results = solve_math_program(m, solver='gurobi')

Read LP format model from file /tmp/tmph5_8jd8h.pyomo.lp
Reading time = 0.00 seconds
x9: 14 rows, 9 columns, 33 nonzeros
Optimize a model with 14 rows, 9 columns and 33 nonzeros
Variable types: 1 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [5e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 14 rows and 9 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 20 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+01, best bound 2.000000000000e+01, gap 0.0000%


In [63]:
model.obj.expr()

20.0

In [60]:
[print(i, m.x[i].value) for i in m.x if m.x[i].value > 0]

(0, 0, 0) 1.0
(0, 1, 0) 1.0


[None, None]

In [71]:
# LP Approximation
env_name = 'Knapsack-v2'
env = gym.make(env_name)
env.item_values = np.array([10, 5])
env.item_probs = np.array([0.75, 0.25])
env.item_weights = np.array([2, 1])
env.N = 2
env.item_numbers = np.arange(env.N)
env.step_limit = 2

# Initialize model
m = ConcreteModel()

# Sets, parameters, and variables
m.W = env.max_weight
m.T = env.step_limit

m.i = Set(initialize=env.item_numbers)
m.t = RangeSet(0, m.T - 1) # Time steps
m.s = Set(initialize=env.item_numbers) # Scenarios

m.w = Param(m.i, 
    initialize={i: j for i, j in zip(env.item_numbers, env.item_weights)})
m.v = Param(m.i, 
    initialize={i: j for i, j in zip(env.item_numbers, env.item_values)})
m.p = Param(m.i,
    initialize={i: j for i, j in zip(env.item_numbers, env.item_probs)})

m.x = Var(m.i, m.t, within=NonNegativeReals)

@m.Constraint()
def weight_constraint(m):
    return sum(m.w[i] * m.x[i, t] * m.p[i]
               for i in m.i
               for t in m.t) - m.W <= 0

@m.Constraint(m.t)
def assignment_constraint(m, t):
    return sum(m.x[i, t]
               for i in m.i) <= 1

# @m.Constraint(m.i, m.t)
# def scenario_constraints(m, i, t):
#     if i == s:
#         return (m.x[i, t, s] <= 1)
#     else:
#         return (m.x[i, t, s] == 0)


m.obj = Objective(expr=(
    sum([m.v[i] * m.x[i, t] * m.p[i]
         for i in m.i
         for t in m.t])),
    sense=maximize)

model, results = solve_math_program(m, 'gurobi')
print(model.obj.expr())

Read LP format model from file /tmp/tmpd4exmg_i.pyomo.lp
Reading time = 0.00 seconds
x5: 4 rows, 5 columns, 9 nonzeros
Optimize a model with 4 rows, 5 columns and 9 nonzeros
Coefficient statistics:
  Matrix range     [2e-01, 2e+00]
  Objective range  [1e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 4 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  1.500000000e+01
15.0


In [72]:
[m.x[i].value for i in m.x]

[1.0, 1.0, 0.0, 0.0]